In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Caminhos de entrada e saída
base_path = "./processed_database"
output_representation_csv = "representation_base.csv"
output_description_csv = "description_base.csv"

# Função para segmentação e limiarização
def segment_image(image):
    """Segmenta a imagem binarizando-a para destacar o objeto com fundo branco."""
    _, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Remoção de pequenos ruídos
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    return binary

# Função para gerar código de cadeias
def chain_code(contour):
    """Gera a representação por código de cadeias do contorno (diferencial)."""
    chain = []
    directions = [(0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1)]
    prev_point = contour[0][0]
    for point in contour[1:]:
        diff = point[0] - prev_point
        normalized_diff = tuple(np.sign(diff))
        if normalized_diff in directions:
            chain.append(directions.index(normalized_diff))
        prev_point = point[0]
    return chain

# Função para gerar assinatura do contorno
def contour_signature(contour):
    """Gera a assinatura do contorno normalizada."""
    M = cv2.moments(contour)
    if M["m00"] == 0:
        return []
    cx = int(M["m10"] / M["m00"])
    cy = int(M["m01"] / M["m00"])
    distances = [np.sqrt((point[0][0] - cx) ** 2 + (point[0][1] - cy) ** 2) for point in contour]
    max_distance = max(distances) if distances else 1
    return [d / max_distance for d in distances]

# Função para calcular descritores de fronteira e região
def calculate_descriptors(contour, binary):
    """Calcula os descritores de fronteira e região para o contorno."""
    perimeter = cv2.arcLength(contour, True)
    area = cv2.contourArea(contour)
    bounding_box = cv2.boundingRect(contour)
    circularity = (4 * np.pi * area) / (perimeter ** 2) if perimeter > 0 else 0

    # Momentos de Hu
    moments = cv2.moments(contour)
    hu_moments = cv2.HuMoments(moments).flatten()

    # Razão de área convexa
    convex_hull = cv2.convexHull(contour)
    convex_area = cv2.contourArea(convex_hull)
    solidity = area / convex_area if convex_area > 0 else 0

    # Eccentricidade
    if len(contour) >= 5:
        ellipse = cv2.fitEllipse(contour)
        eccentricity = np.sqrt(1 - (ellipse[1][1] / ellipse[1][0]) ** 2) if ellipse[1][0] > ellipse[1][1] else 0
    else:
        eccentricity = 0

    descriptors = {
        "Perimeter": perimeter,
        "Area": area,
        "BoundingBoxRatio": bounding_box[2] / bounding_box[3] if bounding_box[3] > 0 else 0,
        "Circularity": circularity,
        "Solidity": solidity,
        "Eccentricity": eccentricity,
        "HuMoments": hu_moments.tolist()
    }
    return descriptors

# Processar imagens por categoria
categories = ["maca", "banana", "laranja"]
representation_data = []
description_data = []

for category in categories:
    input_folder = os.path.join(base_path, category)

    for image_file in sorted(os.listdir(input_folder)):
        if image_file.endswith(".jpg"):
            # Caminho completo da imagem
            image_path = os.path.join(input_folder, image_file)

            # Leitura da imagem
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            # Segmentação
            binary = segment_image(image)

            # Encontrar contornos
            contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if contours:
                largest_contour = max([cnt for cnt in contours if cv2.contourArea(cnt) > 100], 
                                      key=cv2.contourArea, default=None)

                if largest_contour is not None:
                    # Representação
                    chain = chain_code(largest_contour)
                    signature = contour_signature(largest_contour)
                    representation_data.append({
                        "Image": image_file,
                        "Category": category,
                        "ChainCode": chain,
                        "Signature": signature
                    })

                    # Descritores
                    descriptors = calculate_descriptors(largest_contour, binary)
                    description_data.append({
                        "Image": image_file,
                        "Category": category,
                        "Perimeter": descriptors["Perimeter"],
                        "Area": descriptors["Area"],
                        "BoundingBoxRatio": descriptors["BoundingBoxRatio"],
                        "Circularity": descriptors["Circularity"],
                        "Solidity": descriptors["Solidity"],
                        "Eccentricity": descriptors["Eccentricity"],
                        "HuMoment1": descriptors["HuMoments"][0],
                        "HuMoment2": descriptors["HuMoments"][1],
                        "HuMoment3": descriptors["HuMoments"][2]
                    })

# Salvar bases de dados em CSV
representation_df = pd.DataFrame(representation_data)
representation_df.to_csv(output_representation_csv, index=False)

description_df = pd.DataFrame(description_data)
description_df.to_csv(output_description_csv, index=False)

print(f"Bases de dados salvas: \n - {output_representation_csv} \n - {output_description_csv}")


Bases de dados salvas: 
 - representation_base.csv 
 - description_base.csv
